<a href="https://colab.research.google.com/github/aashish513/chatPDF/blob/main/chatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retriever Augmented Generation (RAG)

**Idea** Provide an LLM with access to a specific document file, such that it produces answer based explicitly on the provided document.

**Motivation** There are several reasons why an RAG system may be used. Most prominent ones include the following-
- Bypass an LLM's inherent Token context window
- Give the LLM access to your knowledge, without explicitly training the LLM on your data



## INSTALLATIONS

In [1]:
!pip install tiktoken chromadb pypdf #sentence_transformers InstructorEmbedding faiss-cpu
!pip install langchain -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.6 MB/s eta 0:00:

In [2]:
!pip install -U pysqlite3-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [9]:
!pip install langchain-community langchain-core

  Using cached langchain_community-0.2.10-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


## SECRET KEY DECLARATION AND IMPORTS

In [5]:
import os



In [7]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [ ]:
# import google.generativeai as genai
# import PIL.Image
# import os



# genai.configure(api_key=GEMINI_API_KEY)
# # img = PIL.Image.open('path/to/image.png')

# model = genai.GenerativeModel(model_name="gemini-1.5-flash")
# response = model.generate_content(["You There?"])
# print(response.text)

In [10]:
import numpy as np
import os
import time
from PIL import Image
import random
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, initialize_agent
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
import pandas as pd
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import re

In [11]:
%pip install --upgrade --quiet  langchain-google-genai pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.3 MB/s eta 0:00:00


## RAG DEFINITION

To explore
- Temperature
- Model
- Chunking strategy
- Search strategy

In [13]:
!pip freeze | grep langchain

langchain==0.2.11
langchain-community==0.2.10
langchain-core==0.2.24
langchain-google-genai==1.0.8
langchain-text-splitters==0.2.2


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

# If there is no env variable set for API key, you can pass the API key
# to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                 temperature=0.7, top_p=0.85)

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)

In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/1706.03762")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048) #CharacterTextSplitter(chunk_size=2048, chunk_overlap=128) #   , streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
texts = loader.load_and_split(text_splitter=text_splitter)

# texts = text_splitter.split_documents(documents)

In [17]:
len(texts)

27

In [18]:
for text in texts:
    print(len(text.page_content))

1956
1008
2033
2040
584
1826
1968
639
2025
1253
2027
1556
2037
1462
2033
1252
1962
1163
1979
1289
1978
1392
2015
1405
812
814
817


In [20]:
print(type(texts))
print(len(texts))

<class 'list'>
27


In [21]:
texts[1]

Document(metadata={'source': 'https://arxiv.org/pdf/1706.03762', 'page': 0}, page_content='the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and\nhas been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and\nefficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating\nour research.\n†Work performed while at Google Brain.\n‡Work performed while at Google Research.\n31st Conference on N

In [22]:
texts[1].page_content

'the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and\nhas been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and\nefficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating\nour research.\n†Work performed while at Google Brain.\n‡Work performed while at Google Research.\n31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v7  

In [23]:
for i in range(len(texts)):
  print(len(texts[i].page_content))

1956
1008
2033
2040
584
1826
1968
639
2025
1253
2027
1556
2037
1462
2033
1252
1962
1163
1979
1289
1978
1392
2015
1405
812
814
817


searching in chunks
- Similarity
- MMR (Maximum Maeginal Relevance)

In [25]:
persist_dir = 'txt_embeds'

docsearch = Chroma.from_documents(texts, doc_embeddings, persist_directory=persist_dir)

qa = RetrievalQA.from_chain_type(llm = llm, chain_type='stuff', retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={'k': 5}, return_source_documents = True))

In [29]:
qa("what topics are covered")

{'query': 'what topics are covered',
 'result': 'The provided text discusses the following topics:\n\n* **Neural sequence transduction models:**  These models are used for tasks like machine translation, where an input sequence is transformed into an output sequence. \n* **Encoder-decoder structure:** A common architecture for these models, where an encoder maps the input sequence to a representation, and a decoder generates the output sequence based on this representation.\n* **Self-attention:** A mechanism that allows the model to attend to different parts of the input sequence when generating the output.\n* **Multi-head attention:** An extension of self-attention that uses multiple attention heads to capture different aspects of the input.\n* **Training details:** The text describes the training process, including the dataset used, hardware, optimizer, and regularization techniques. \n'}

In [31]:
qa("Who wrote this paper")["result"]

"The provided text is a snippet from a research paper on neural machine translation. Unfortunately, the paper's authors are not explicitly mentioned in this excerpt. \n"

In [ ]:
while True:
  response = qa(input("User: "))
  print("LLM: ", response["result"], "\n\n")


User: Hello what topics are discussed in the paper
LLM:  The paper discusses the architecture of a neural machine translation model that utilizes self-attention. It delves into the following topics:

* **Encoder-Decoder Structure:**  The paper explains the common encoder-decoder structure used in sequence transduction models, where the encoder maps input sequences to continuous representations and the decoder generates output sequences based on these representations.
* **Multi-Head Attention:** The paper introduces the concept of multi-head attention, which enhances the attention mechanism by performing multiple attention functions in parallel with different learned projections.
* **Position-wise Feed-Forward Networks:** The paper describes the use of fully connected feed-forward networks, applied to each position separately, as part of the encoder and decoder layers.
* **Embeddings and Softmax:** The paper explains how learned embeddings are used to convert input and output tokens int